In [1]:
# Import des librairies utilisées
import polars as pl
from pymongo import MongoClient

---
**Récupération des données à partir d'URLs**
-
---

In [4]:
data_url = 'https://s3.eu-west-1.amazonaws.com/course.oc-static.com/projects/922_Data+Engineer/922_P7/listings_Paris+(1).csv'
df = pl.read_csv(data_url)
df_paris = df.filter(pl.col("host_location") == "Paris, France")
print(df_paris)

shape: (66_031, 75)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ id        ┆ listing_u ┆ scrape_id ┆ last_scra ┆ … ┆ calculate ┆ calculate ┆ calculate ┆ reviews_ │
│ ---       ┆ rl        ┆ ---       ┆ ped       ┆   ┆ d_host_li ┆ d_host_li ┆ d_host_li ┆ per_mont │
│ i64       ┆ ---       ┆ i64       ┆ ---       ┆   ┆ stings_co ┆ stings_co ┆ stings_co ┆ h        │
│           ┆ str       ┆           ┆ str       ┆   ┆ unt…      ┆ unt…      ┆ unt…      ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ ---       ┆ ---       ┆ ---       ┆ f64      │
│           ┆           ┆           ┆           ┆   ┆ i64       ┆ i64       ┆ i64       ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 80260     ┆ https://w ┆ 202406101 ┆ 2024-06-1 ┆ … ┆ 1         ┆ 0         ┆ 0         ┆ 1.29     │
│           ┆ ww.airbnb ┆ 95007     ┆ 3         ┆   ┆           ┆      

---
**Création de la base de données**
-
---

In [7]:
# Connexion à MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["mydb"]
collection = db["annonces"]

---
**Import des données dans la base**
-
---

In [10]:
data = df_paris.to_dicts()
collection.insert_many(data)
print(f"{len(data)} documents insérés")

66031 documents insérés


---
Q1: Nombre de documents
-
---

In [12]:
print(collection.count_documents({}))

66031


---
Q2: Nombre de logements ayant des disponibilités
-
---

In [14]:
values = collection.distinct('has_availability')
print(values)
print(collection.count_documents({'has_availability': 't'}))

[None, 'f', 't']
62581


---
Q3: Nombre d’annonces par type de location
-
---

In [17]:
# Pipeline d'agrégation
pipeline = [
    {"$group": {"_id": "$room_type", "nombre_annonces": {"$sum": 1}}},
    {"$sort": {"nombre_annonces": -1}}  # tri décroissant
]

result = collection.aggregate(pipeline)

for doc in result:
    print(f"{doc['_id']}: {doc['nombre_annonces']} annonces")

Entire home/apt: 59201 annonces
Private room: 6067 annonces
Hotel room: 513 annonces
Shared room: 250 annonces


---
Q4: Nombre d’annonces par type de location
-
---

In [19]:
# Pipeline d'agrégation
pipeline = [
    {"$group": {"_id": "$property_type", "nombre_annonces": {"$sum": 1}}},
    {"$sort": {"nombre_annonces": -1}}  # tri décroissant
]

result = collection.aggregate(pipeline)

for doc in result:
    print(f"{doc['_id']}: {doc['nombre_annonces']} annonces")

Entire rental unit: 55483 annonces
Private room in rental unit: 4252 annonces
Entire condo: 1862 annonces
Entire loft: 816 annonces
Room in boutique hotel: 789 annonces
Entire home: 478 annonces
Private room in bed and breakfast: 430 annonces
Room in hotel: 389 annonces
Private room in condo: 268 annonces
Entire townhouse: 215 annonces
Shared room in rental unit: 179 annonces
Entire serviced apartment: 177 annonces
Private room in home: 94 annonces
Private room in townhouse: 76 annonces
Private room in loft: 68 annonces
Private room in hostel: 44 annonces
Shared room in hostel: 39 annonces
Entire place: 32 annonces
Private room: 31 annonces
Private room in guesthouse: 29 annonces
Private room in guest suite: 29 annonces
Room in aparthotel: 27 annonces
Private room in casa particular: 23 annonces
Tiny home: 21 annonces
Entire guesthouse: 19 annonces
Entire vacation home: 18 annonces
Entire guest suite: 17 annonces
Room in serviced apartment: 13 annonces
Entire villa: 12 annonces
Private

---
Q5: Les 5 annonces de location avec le plus d’évaluations et leurs nombre
-
---

In [21]:
top5 = collection.find(
    {},  # pas de filtre, on prend toutes les annonces
    {"name": 1, "number_of_reviews": 1, "_id": 0}  # projection: colonnes à afficher
    ).sort("number_of_reviews", -1).limit(5)

for doc in top5:
    print(f"{doc['name']}: {doc['number_of_reviews']} reviews")

Sweet & cosy room next to Canal Saint Martin ❤️: 3067 reviews
Double/Twin Room, close to Opera and the Louvre with breakfast included: 2620 reviews
Bed in Dorm of 8 Beds "The Big One" in Paris: 2294 reviews
Comfortable bed in shared rooms of 8 in Paris 12e: 2105 reviews
Hotel Boronali *** - Double room in Montmartre: 1716 reviews


---
Q6: Nombre total d’hôtes différents
-
---

In [28]:
# Pipeline d'agrégation
pipeline = [
    {"$group": {"_id": "$host_id"}},
    {"$count": "total_hosts"}  # tri décroissant
]
result = collection.aggregate(pipeline)
for doc in result:
    print(doc)

{'total_hosts': 50772}


---
Q7: Nombre de locations réservables instantanément et la proportion
-
---

In [31]:
# Pipeline d'agrégation
pipeline = [
    {
        "$group": {
            "_id": None,
            "total": {"$sum": 1},
            "instant": {"$sum": {"$cond": [{"$eq": ["$instant_bookable", "t"]}, 1, 0]}}
        }
    },
    {
        "$project": {
            "_id": 0,
            "total": 1,
            "instant": 1,
            "proportion": {"$divide": ["$instant", "$total"]}
        }
    }
]
result = collection.aggregate(pipeline)
for doc in result:
    print(doc)

{'total': 66031, 'instant': 12598, 'proportion': 0.190789174781542}


---
Q8: Nombre d’hôtes ayant plus de 100 annonces sur les plateformes, leurs noms et la proportion
-
---

In [34]:
pipeline = [
    # Grouper par hôte pour compter le nombre d'annonces
    {
        "$group": {
            "_id": {"host_id": "$host_id", "host_name": "$host_name"},
            "nb_annonces": {"$sum": 1}
        }
    },
    # Filtrer les hôtes avec plus de 100 annonces
    {"$match": {"nb_annonces": {"$gt": 100}}},
    # Trier par nombre d'annonces décroissant
    {"$sort": {"nb_annonces": -1}},
    # Calculer la somme totale d'annonces pour ce sous-ensemble
    {
        "$group": {
            "_id": None,
            "total_annonces": {"$sum": "$nb_annonces"},
            "hotes": {"$push": {"host_id": "$_id.host_id", "host_name": "$_id.host_name", "nb_annonces": "$nb_annonces"}}
        }
    },
    # Déplier la liste pour calculer la proportion par hôte
    {"$unwind": "$hotes"},
    {
        "$addFields": {
            "hotes.proportion": {"$divide": ["$hotes.nb_annonces", "$total_annonces"]}
        }
    },
    # Reformater la liste finale
    {
        "$group": {
            "_id": None,
            "hotes": {"$push": "$hotes"},
            "total_annonces": {"$first": "$total_annonces"}
        }
    }
]

result = list(collection.aggregate(pipeline))

# Affichage
if result:
    r = result[0]
    print(f"Total d'annonces pour hôtes >100 annonces : {r['total_annonces']}\n")
    print("Liste des hôtes avec proportion respective :")
    for h in r['hotes']:
        print(f"\t{h['host_name']} : {h['nb_annonces']} annonces, proportion : {h['proportion']:.2%}")

Total d'annonces pour hôtes >100 annonces : 3279

Liste des hôtes avec proportion respective :
	Blueground : 730 annonces, proportion : 22.26%
	Pierre De WeHost : 426 annonces, proportion : 12.99%
	Cédric De ClickYourFlat : 274 annonces, proportion : 8.36%
	Ludovic : 211 annonces, proportion : 6.43%
	David Et Warren : 210 annonces, proportion : 6.40%
	Welkeys : 170 annonces, proportion : 5.18%
	Studioprestige : 154 annonces, proportion : 4.70%
	Sweet Inn : 149 annonces, proportion : 4.54%
	IntoParis : 145 annonces, proportion : 4.42%
	Parisian Home : 132 annonces, proportion : 4.03%
	Barnes : 122 annonces, proportion : 3.72%
	Jérémy : 120 annonces, proportion : 3.66%
	Michael & Johanna : 119 annonces, proportion : 3.63%
	Giacomo : 111 annonces, proportion : 3.39%
	Checkmyguest : 104 annonces, proportion : 3.17%
	Pierre : 102 annonces, proportion : 3.11%


---
Q9: Nombre de super hôtes différents et la proportion globale
-
---

In [37]:
pipeline = [
    # Grouper par host_id pour obtenir un hôte unique
    {
        "$group": {
            "_id": "$host_id",
            "host_is_superhost": {"$first": "$host_is_superhost"}
        }
    },
    # Grouper pour compter superhosts et total
    {
        "$group": {
            "_id": None,
            "total_hosts": {"$sum": 1},
            "superhosts": {
                "$sum": {
                    "$cond": [{"$eq": ["$host_is_superhost", "t"]}, 1, 0]
                }
            }
        }
    },
    # Ajouter la proportion
    {
        "$addFields": {
            "proportion": {"$divide": ["$superhosts", "$total_hosts"]}
        }
    }
]

result = list(collection.aggregate(pipeline))

if result:
    r = result[0]
    print(f"Nombre de super-hôtes : {r['superhosts']}")
    print(f"Nombre total d'hôtes : {r['total_hosts']}")
    print(f"Proportion globale : {r['proportion']:.2%}")

Nombre de super-hôtes : 7509
Nombre total d'hôtes : 50772
Proportion globale : 14.79%


---
Q10: Taux de réservation moyen par mois par type de logement
-
---

In [40]:
result = (
    df_paris.group_by("room_type")
    .agg(pl.col("reviews_per_month").mean().alias("avg_reviews_per_month"))
    .sort("avg_reviews_per_month", descending=True)
)
print(result)

shape: (4, 2)
┌─────────────────┬───────────────────────┐
│ room_type       ┆ avg_reviews_per_month │
│ ---             ┆ ---                   │
│ str             ┆ f64                   │
╞═════════════════╪═══════════════════════╡
│ Shared room     ┆ 2.201517              │
│ Private room    ┆ 1.329301              │
│ Hotel room      ┆ 1.152362              │
│ Entire home/apt ┆ 1.039023              │
└─────────────────┴───────────────────────┘


---
Q11: Taux de réservation moyen par mois par type de logement
-
---

In [43]:
result = (
    df_paris
    .filter(pl.col("reviews_per_month").is_not_null())
    .group_by("property_type")
    .agg(pl.col("reviews_per_month").mean().alias("avg_reviews_per_month"))
    .sort("avg_reviews_per_month", descending=True)
)
print(result)

shape: (55, 2)
┌───────────────────────────────┬───────────────────────┐
│ property_type                 ┆ avg_reviews_per_month │
│ ---                           ┆ ---                   │
│ str                           ┆ f64                   │
╞═══════════════════════════════╪═══════════════════════╡
│ Room in hostel                ┆ 5.883333              │
│ Private room in boat          ┆ 5.34                  │
│ Shared room in hostel         ┆ 4.607632              │
│ Shared room in boutique hotel ┆ 3.6                   │
│ Private room in vacation home ┆ 3.36                  │
│ …                             ┆ …                     │
│ Private room in villa         ┆ 0.45                  │
│ Shared room in farm stay      ┆ 0.36                  │
│ Entire villa                  ┆ 0.316                 │
│ Cave                          ┆ 0.25                  │
│ Earthen home                  ┆ 0.2                   │
└───────────────────────────────┴───────────────────────┘

---
Q12: Médiane du nombre d’avis pour tous les logements
-
---

In [46]:
median_reviews = df_paris.select(pl.col("number_of_reviews").median()).item()
print(median_reviews)

4.0


---
Q13: Médiane du nombre d’avis par catégorie d’hôte
-
---

In [53]:
result = (
    df_paris
    .group_by("host_is_superhost")
    .agg(pl.col("number_of_reviews").median().alias("median_reviews"))
)
print(result)

shape: (3, 2)
┌───────────────────┬────────────────┐
│ host_is_superhost ┆ median_reviews │
│ ---               ┆ ---            │
│ str               ┆ f64            │
╞═══════════════════╪════════════════╡
│ t                 ┆ 27.0           │
│ f                 ┆ 3.0            │
│ null              ┆ 15.0           │
└───────────────────┴────────────────┘


---
Q14: Densité de logements par quartier de Paris
-
---

In [63]:
pipeline = [
    {"$group": {"_id": "$neighbourhood_cleansed", "nombre_annonces": {"$sum": 1}}},
    {"$sort": {"nombre_annonces": -1}}  # tri décroissant
]
result = list(collection.aggregate(pipeline))
print(result)

[{'_id': 'Buttes-Montmartre', 'nombre_annonces': 7772}, {'_id': 'Popincourt', 'nombre_annonces': 6141}, {'_id': 'Vaugirard', 'nombre_annonces': 5030}, {'_id': 'Entrepôt', 'nombre_annonces': 4829}, {'_id': 'Batignolles-Monceau', 'nombre_annonces': 4584}, {'_id': 'Buttes-Chaumont', 'nombre_annonces': 4135}, {'_id': 'Ménilmontant', 'nombre_annonces': 4067}, {'_id': 'Passy', 'nombre_annonces': 3872}, {'_id': 'Opéra', 'nombre_annonces': 3108}, {'_id': 'Temple', 'nombre_annonces': 2726}, {'_id': 'Reuilly', 'nombre_annonces': 2709}, {'_id': 'Observatoire', 'nombre_annonces': 2367}, {'_id': 'Gobelins', 'nombre_annonces': 2174}, {'_id': 'Bourse', 'nombre_annonces': 2045}, {'_id': 'Hôtel-de-Ville', 'nombre_annonces': 1972}, {'_id': 'Panthéon', 'nombre_annonces': 1927}, {'_id': 'Élysée', 'nombre_annonces': 1796}, {'_id': 'Luxembourg', 'nombre_annonces': 1746}, {'_id': 'Palais-Bourbon', 'nombre_annonces': 1705}, {'_id': 'Louvre', 'nombre_annonces': 1326}]


---
Q15: Quartiers avec le plus fort taux de réservation par mois
-
---

In [68]:
result = (
    df_paris
    .group_by("neighbourhood_cleansed")
    .agg(pl.col("reviews_per_month").mean().alias("avg_reviews_per_month"))
    .sort("avg_reviews_per_month", descending=True)
)
print(result)

shape: (20, 2)
┌────────────────────────┬───────────────────────┐
│ neighbourhood_cleansed ┆ avg_reviews_per_month │
│ ---                    ┆ ---                   │
│ str                    ┆ f64                   │
╞════════════════════════╪═══════════════════════╡
│ Bourse                 ┆ 1.565653              │
│ Louvre                 ┆ 1.35715               │
│ Élysée                 ┆ 1.284484              │
│ Hôtel-de-Ville         ┆ 1.237135              │
│ Palais-Bourbon         ┆ 1.210721              │
│ …                      ┆ …                     │
│ Popincourt             ┆ 0.9992                │
│ Buttes-Montmartre      ┆ 0.974086              │
│ Batignolles-Monceau    ┆ 0.973816              │
│ Buttes-Chaumont        ┆ 0.833195              │
│ Ménilmontant           ┆ 0.826698              │
└────────────────────────┴───────────────────────┘


---
**Nettoyage**

---

In [130]:
# Fermeture de la connexion DB
client.close()